5 basic steps for Data analysis:
             
1. Define the problem
2. Collect data
3. Prepare data
4. Analyse data
5. Interpret results

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

import builtins
import rich
builtins.print = rich.print

In [ ]:
df = pd.read_csv("../data/netflix_titles.csv")

## Task 1: Transform the initial table

- Initially, each row is for one movie, available in potentiall MANY countries.
- For easier analsys concerning country, we want each row for a movie in a single country
- In other words, one row of the initial table corresponds to multiple rows in the new table

## Task 2: List all TV-Shows

Given the dataset, I will filter out by column type

In [ ]:
df_tv_show = df.query('type == "TV Show"')
df_tv_show.head()

## Task 3: How many movies and TV-shows are available in each country?

Target result: a bar plot:
- x-axis: country name
- y-axis: no. movies and TV-shows

I want a table with 2 columns
1. country
2. Count of movies and TV-shows available in that country

Approach: create a dictionary with
key: country name
value: count of movies and tv-shows

In [ ]:
# Remove nan values
df = df.dropna()

# A dictionary
# key: country name
# value: count of movies and tv-shows
dict_country_cnt = {}

# Iterate through the array df.country.unique()
for id_country, country_names in enumerate(df.country.unique()):
    # If country_names == nan, skip
    if isinstance(country_names, float):
        continue

    # If it is available in many countries, split the string
    for country_name in country_names.split(', '):
        # If the country_name is already in dict_country_cnt.keys(), skip
        if country_name in list(dict_country_cnt.keys()):
            continue
        # else, save it in the list
        else:
            dict_country_cnt[country_name] = 0

# For each country, count movies that are available in that country
for country_name in dict_country_cnt.keys():
    filtered_df = df[df['country'].str.contains(country_name)]
    dict_country_cnt[country_name] = len(filtered_df.index)

In [ ]:
del dict_country_cnt['']

dict_country_cnt = sorted(dict_country_cnt.items(), key=lambda x:x[1], reverse=True)
dict_country_cnt = dict(dict_country_cnt)

# Prepare the data
df_new = pd.DataFrame(dict_country_cnt.items(), columns=['country', 'cnt'])

sns.set_theme(rc={'figure.figsize':(7,22)})
sns.barplot(df_new, y="country", x='cnt')

plt.savefig('../results/barplot_country_no_movies.png')

plt.show()
plt.close()

## Task 4: In the top 5 countries with the highest number of available movies and TV-shows, see the percentage (or number) of movies versus TV-shows

## Task 5: In USA, is there any trend on no. movies & TV-shows release over the years

## Task 6: In USA, is there any trend on the duration of movies & TV-shows, considering the released years?